In [1]:
import pandas as pd
import numpy as np

In [2]:
order_list = pd.read_csv('output/order_list_sampled.csv')
freight_rates = pd.read_csv('input/freight_rates.csv')
wh_costs = pd.read_csv('input/wh_costs.csv')

In [3]:
order_list['Carrier'] = order_list['Carrier'].fillna(0).astype(int)
order_list

,Order ID,Origin Port,Carrier,Service Level,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight,Maximum delivery time
0,1755,4,1,1,21,1198,3,1,1287,5.828514,7
1,1522,4,1,1,32,259,3,1,880,9.360000,7
2,5532,4,1,1,15,920,3,1,1256,18.420000,4
3,8228,4,1,1,35,235,3,1,363,0.348672,4
4,3840,4,2,1,2,394,3,1,1276,2.641625,4
...,...,...,...,...,...,...,...,...,...,...,...
995,4025,4,2,1,10,995,3,1,311,1.384804,8
996,8643,4,1,1,16,1197,3,1,294,0.239737,4
997,5939,4,1,1,34,50,3,1,1005,3.620000,7
998,7317,4,2,1,26,27,3,1,618,14.044822,3


In [90]:
freight_rates

,Carrier,orig_port_cd,dest_port_cd,mode_dsc,max_wgh_qty,minm_wgh_qty,svc_cd,minimum cost,rate,tpt_day_cnt
0,1,2,1,0,249.99,100.0,1,23.8384,0.0844,4
1,1,2,1,0,499.99,250.0,1,23.8384,0.0808,4
2,1,2,1,0,1999.99,500.0,1,23.8384,0.0784,4
3,1,2,1,0,99999.99,2000.0,1,23.8384,0.0784,4
4,1,3,1,1,5000.00,0.0,1,28.9442,12.6942,1
...,...,...,...,...,...,...,...,...,...,...
96,9,8,1,0,499.99,0.0,1,231.3072,0.1880,14
97,9,8,1,0,999.99,500.0,1,231.3072,0.1224,14
98,9,8,1,0,4999.99,1000.0,1,231.3072,0.1116,14
99,9,8,1,0,9999.99,5000.0,1,231.3072,0.1036,14


In [91]:
# rearrange the freight_rates table
freight_rates_rearranged = freight_rates.groupby(['Carrier', 'orig_port_cd', 'dest_port_cd', 'mode_dsc']).max('rate').reset_index()
freight_rates_rearranged = freight_rates_rearranged.loc[:, ['Carrier', 'orig_port_cd', 'dest_port_cd', 'mode_dsc', 'rate']]
freight_rates_rearranged

,Carrier,orig_port_cd,dest_port_cd,mode_dsc,rate
0,1,2,1,0,0.084400
1,1,3,1,1,12.694200
2,1,4,1,0,0.049400
3,1,9,1,1,12.778400
4,2,2,1,0,0.088800
5,2,4,1,0,0.081200
6,2,5,1,0,0.128346
7,2,6,1,0,0.090800
8,2,10,1,0,0.097800
9,3,2,1,0,0.077200


In [92]:
# create ground dict
ground_freight_rates = freight_rates_rearranged[freight_rates_rearranged['mode_dsc'] == 1]
ground_freight_rates = ground_freight_rates.reset_index(drop=True)

ground_dict = dict()
for i in range(len(ground_freight_rates)):
    ground_dict[(ground_freight_rates.loc[i, 'Carrier'].astype(int),
                 ground_freight_rates.loc[i, 'orig_port_cd'].astype(int),
                 ground_freight_rates.loc[i, 'dest_port_cd'].astype(int))] = ground_freight_rates.loc[i, 'rate']
ground_dict

{(1, 3, 1): 12.6942,
 (1, 9, 1): 12.7784,
 (5, 3, 1): 13.610866666666666,
 (5, 9, 1): 20.7784,
 (7, 3, 1): 128.02720000000002}

In [107]:
(1, 3, 1) in ground_dict.keys()

True

In [93]:
# create air dict
air_freight_rates = freight_rates_rearranged[freight_rates_rearranged['mode_dsc'] == 0]
air_freight_rates = air_freight_rates.reset_index(drop=True)

air_dict = dict()
for i in range(len(air_freight_rates)):
    air_dict[(air_freight_rates.loc[i, 'Carrier'].astype(int),
                 air_freight_rates.loc[i, 'orig_port_cd'].astype(int),
                 air_freight_rates.loc[i, 'dest_port_cd'].astype(int))] = air_freight_rates.loc[i, 'rate']
air_dict

{(1, 2, 1): 0.0843999999999999,
 (1, 4, 1): 0.0494,
 (2, 2, 1): 0.0888,
 (2, 4, 1): 0.0812,
 (2, 5, 1): 0.128346,
 (2, 6, 1): 0.0908,
 (2, 10, 1): 0.0978,
 (3, 2, 1): 0.0772,
 (3, 7, 1): 0.2094,
 (3, 8, 1): 0.1197333333333333,
 (3, 10, 1): 0.1941,
 (3, 11, 1): 0.0797333333333333,
 (4, 2, 1): 0.096,
 (4, 4, 1): 0.1024,
 (4, 5, 1): 0.1352,
 (4, 6, 1): 0.096,
 (4, 10, 1): 0.1180305261115716,
 (5, 2, 1): 0.06392,
 (5, 5, 1): 0.0956,
 (5, 6, 1): 0.16192,
 (6, 8, 1): 0.7132,
 (6, 10, 1): 0.7132,
 (8, 2, 1): 0.0928,
 (8, 3, 1): 0.2893,
 (8, 4, 1): 0.1322,
 (8, 9, 1): 0.0482,
 (8, 11, 1): 0.1592,
 (9, 8, 1): 0.1879999999999999}

In [109]:
# calculate ground transportation cost
ground_index_flag = np.zeros(len(ground_dict))
list_combination = []
for i in range(len(order_list)):
    list_combination.append((order_list.loc[i, 'Carrier'], order_list.loc[i, 'Origin Port'], order_list.loc[i, 'Destination Port']))
    if (order_list.loc[i, 'Carrier'], order_list.loc[i, 'Origin Port'], order_list.loc[i, 'Destination Port']) in ground_dict.keys():
        ground_dict[(order_list.loc[i, 'Carrier'], order_list.loc[i, 'Origin Port'], order_list.loc[i, 'Destination Port'])] = 1
    else:
        continue

list_combination = set(list_combination)
list_combination

{(0, 4, 1), (0, 9, 1), (1, 4, 1), (1, 9, 1), (2, 4, 1)}

In [ ]:
# Data conflict
# Currently some orders starts use carrier 1 and start from port 9, which does not appear in freight rate sheet.